In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import gdown
import zipfile
import os

In [2]:
# Loading main data from GDrive.
file_id = "1MNRptIrru5k8FJpBek1gmBK-codD1BDl"
url = f"https://drive.google.com/uc?id={file_id}"
file_extension = "downloaded_file.csv"
gdown.download(url, file_extension, quiet=False)
df = pd.read_csv(file_extension, sep=",")

Downloading...
From (original): https://drive.google.com/uc?id=1MNRptIrru5k8FJpBek1gmBK-codD1BDl
From (redirected): https://drive.google.com/uc?id=1MNRptIrru5k8FJpBek1gmBK-codD1BDl&confirm=t&uuid=947992c6-0758-4ecb-abf3-68a7092d85ff
To: C:\Users\Konrad\Desktop\Magisterka\Kod\2024-mgr-sluzba-cywilna\Python\Code\downloaded_file.csv
100%|██████████| 254M/254M [00:03<00:00, 64.6MB/s] 


### Main data preprocessing

In [3]:
merged_bdl = df.copy()

# More intuitive names.
columns_to_rename = {
    "REGON": "REGON_of_district",
    "wynagrodzenia": "bdl_avg_district_salary_per_year",
    "year_announced": "year",
    "promocja": "advertised_institution",
    "kandydaci": "applicants",
    "views": "ad_views",
    "JPT_NAZWA_": "district_name"
}

for key, value in columns_to_rename.items():
    merged_bdl.rename(columns={key: value}, inplace=True)

In [4]:
# Inconsistent data types (for later merge).
merged_bdl['district'] = merged_bdl['district'].astype(str)
merged_bdl['voivodeship'] = merged_bdl['voivodeship'].astype(str)
merged_bdl['year'] = merged_bdl['year'].astype(int)

### BDL data preprocessing


In [5]:
# Importing BDL data.

cena_mieszkan = pd.DataFrame()
mediana_wieku = pd.DataFrame()
saldo_migracji = pd.DataFrame() # dotyczy osób w wieku produkcyjnym
wsp_feminizacji = pd.DataFrame()
wydatki_powiat = pd.DataFrame() # w złotówkach
wynagrodzenia = pd.DataFrame() # w złotówkach

bdl_dfs_list = [
    cena_mieszkan,
    mediana_wieku,
    saldo_migracji,
    wsp_feminizacji,
    wydatki_powiat,
    wynagrodzenia
]

file_ids = {
    "cena_mieszkan": "12UWwTtqRazIEctdzxib34fs8I_-ZUOD7",
    "mediana_wieku": "1D8jvho56cqn5dNGhMPaHujF3tmoQ1L6K",
    "saldo_migracji": "1sVL3iXGzc9D2k38ogErl84mVIzl6SzIP",
    "wsp_feminizacji": "163IRO4q5iU-MdyN3kYXkFHDavpyr353f",
    "wydatki_powiat": "1YMXflzDmozNqxQLpcd6oc1e1vtUVyUtS",
    "wynagrodzenia": "104z3VjCcv66e2L034D-o3vJY2wROJwCY"
}

for i in range(len(file_ids)):
    url = f"https://drive.google.com/uc?id={list(file_ids.values())[i]}"
    file_extension = "downloaded_file.csv"
    gdown.download(url, file_extension, quiet=False)
    bdl_dfs_list[i] = pd.read_csv(file_extension, sep=";", on_bad_lines='skip')

Downloading...
From: https://drive.google.com/uc?id=12UWwTtqRazIEctdzxib34fs8I_-ZUOD7
To: C:\Users\Konrad\Desktop\Magisterka\Kod\2024-mgr-sluzba-cywilna\Python\Code\downloaded_file.csv
100%|██████████| 204k/204k [00:00<00:00, 3.29MB/s]
Downloading...
From: https://drive.google.com/uc?id=1D8jvho56cqn5dNGhMPaHujF3tmoQ1L6K
To: C:\Users\Konrad\Desktop\Magisterka\Kod\2024-mgr-sluzba-cywilna\Python\Code\downloaded_file.csv
100%|██████████| 144k/144k [00:00<00:00, 3.95MB/s]
Downloading...
From: https://drive.google.com/uc?id=1sVL3iXGzc9D2k38ogErl84mVIzl6SzIP
To: C:\Users\Konrad\Desktop\Magisterka\Kod\2024-mgr-sluzba-cywilna\Python\Code\downloaded_file.csv
100%|██████████| 259k/259k [00:00<00:00, 3.73MB/s]
Downloading...
From: https://drive.google.com/uc?id=163IRO4q5iU-MdyN3kYXkFHDavpyr353f
To: C:\Users\Konrad\Desktop\Magisterka\Kod\2024-mgr-sluzba-cywilna\Python\Code\downloaded_file.csv
100%|██████████| 176k/176k [00:00<00:00, 3.07MB/s]
Downloading...
From: https://drive.google.com/uc?id=1YMX

In [6]:
# Removing unnecessary columns.

unique_columns_to_delete = [
    ["Transakcje rynkowe", "Powierzchnia użytkowa lokali mieszkalnych"],
    ["Migracje", "Wiek"],
    ["Ogółem"],
    ["Wskaźniki"],
    ["Wyszczególnienie"],
    ["Płeć"]
]

for i in range(len(bdl_dfs_list)):
    try:
        bdl_dfs_list[i].drop(["Nazwa", "Jednostka miary", "Atrybut"], axis=1, inplace=True)
        bdl_dfs_list[i] = bdl_dfs_list[i].iloc[:, :-1] # the last column has a different name in every df and is always unnecessary.
    except:
        pass

    # Columns unique for single dataframes.
    for columns in unique_columns_to_delete:
        try:
            bdl_dfs_list[i].drop(columns, axis=1, inplace=True)
        except:
            pass

In [7]:
# Renaming columns with the same name across multiple dataframes.
for i in range(len(bdl_dfs_list)):
    bdl_dfs_list[i].rename(columns={"Wartosc": list(file_ids.keys())[i]}, inplace=True)

In [17]:
# Cleaning non-absolute values of salaries.
bdl_dfs_list[5]["wynagrodzenia"] = (
    bdl_dfs_list[5]["wynagrodzenia"]
    .str.replace(",", ".", regex=False)
    .astype(float)
)

bdl_dfs_list[5] = bdl_dfs_list[5][bdl_dfs_list[5]["wynagrodzenia"] >= 2500]

,Kod,Rok,wynagrodzenia
0,0,2017,4527.89
1,0,2018,4834.76
2,0,2019,5181.63
3,0,2020,5523.32
4,0,2021,6001.02
...,...,...,...
5546,3263000,2019,5222.62
5547,3263000,2020,5517.68
5548,3263000,2021,5564.41
5549,3263000,2022,6061.29


### Merge

In [18]:
for i in range(len(bdl_dfs_list)):
    bdl_dfs_list[i]['Kod'] = bdl_dfs_list[i]['Kod'].astype(str)
    
    mask_whole_country = (bdl_dfs_list[i]['Kod'].str[0] == "0")
    mask_whole_voivodeships = ((bdl_dfs_list[i]['Kod'].str[1] == "0") & (bdl_dfs_list[i]['Kod'].str[2] == "0"))
    
    bdl_dfs_list[i] = bdl_dfs_list[i][~mask_whole_country]
    bdl_dfs_list[i] = bdl_dfs_list[i][~mask_whole_voivodeships]

    # For single-digit voivodeships, the code's length is 6.
    mask_length_6 = (bdl_dfs_list[i]['Kod'].str.len() == 6) 
    mask_length_7 = (bdl_dfs_list[i]['Kod'].str.len() == 7)

    # Single-digits districts include 0 and we want to remove it (e.g. '07' instead of '7').
    mask_district_single_digit_length_6 = ((bdl_dfs_list[i]['Kod'].str[1] == "0") & mask_length_6)
    mask_district_non_single_digit_length_6 = ((bdl_dfs_list[i]['Kod'].str[1] != "0") & mask_length_6)
    mask_district_single_length_7 = ((bdl_dfs_list[i]['Kod'].str[2] == "0") & mask_length_7)
    mask_district_non_single_length_7 = ((bdl_dfs_list[i]['Kod'].str[2] != "0") & mask_length_7)
    
    # Districts.
    bdl_dfs_list[i].loc[mask_district_single_digit_length_6, 'district'] = bdl_dfs_list[i].loc[mask_district_single_digit_length_6, 'Kod'].str[2:3]
    bdl_dfs_list[i].loc[mask_district_non_single_digit_length_6, 'district'] = bdl_dfs_list[i].loc[mask_district_non_single_digit_length_6, 'Kod'].str[1:3]
    bdl_dfs_list[i].loc[mask_district_single_length_7, 'district'] = bdl_dfs_list[i].loc[mask_district_single_length_7, 'Kod'].str[3:4]
    bdl_dfs_list[i].loc[mask_district_non_single_length_7, 'district'] = bdl_dfs_list[i].loc[mask_district_non_single_length_7, 'Kod'].str[2:4]
    
    # Voivodeships.
    bdl_dfs_list[i].loc[mask_length_6, 'voivodeship'] = bdl_dfs_list[i].loc[mask_length_6, 'Kod'].str[0:1]
    bdl_dfs_list[i].loc[mask_length_7, 'voivodeship'] = bdl_dfs_list[i].loc[mask_length_7, 'Kod'].str[0:2]

    # Inconsistent data types.
    bdl_dfs_list[i]['district'] = bdl_dfs_list[i]['district'].astype(str)
    bdl_dfs_list[i]['voivodeship'] = bdl_dfs_list[i]['voivodeship'].astype(str)
    bdl_dfs_list[i]['Rok'] = bdl_dfs_list[i]['Rok'].astype(int)

    bdl_dfs_list[i].drop(['Kod'], axis=1, inplace=True)

    # Merge
    merged_bdl = pd.merge(merged_bdl, bdl_dfs_list[i], left_on=["district", "voivodeship", "year"], right_on=["district", "voivodeship", "Rok"], how="left")
    merged_bdl.drop(["Rok"], axis=1, inplace=True)

C:\Users\Konrad\AppData\Local\Temp\ipykernel_4384\1083514685.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bdl_dfs_list[i] = bdl_dfs_list[i][~mask_whole_voivodeships]
C:\Users\Konrad\AppData\Local\Temp\ipykernel_4384\1083514685.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bdl_dfs_list[i] = bdl_dfs_list[i][~mask_whole_voivodeships]
C:\Users\Konrad\AppData\Local\Temp\ipykernel_4384\1083514685.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bdl_dfs_list[i] = bdl_dfs_list[i][~mask_whole_voivodeships]
C:\Users\Konrad\AppData\Local\Temp\ipykernel_4384\1083514685.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bdl_dfs_list[i] = bdl_dfs_list[i][~mask_whole_voivodeships]
C:\Users\Konrad\AppData\Local\Temp\ipykernel_4384\1083514685.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bdl_dfs_list[i] = bdl_dfs_list[i][~